# accessing last layer of birdnet model

- colab link: https://colab.research.google.com/drive/1d9u93LJQQRTgAmzFMeCaFP0U22dUJOl-?usp=sharing

In [4]:
from pathlib import Path
import requests

checkpoint_path = Path("data/raw/birdnet/v2.2/checkpoint")

checkpoint_base_url = "https://github.com/kahst/BirdNET-Analyzer/raw/main/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model"
for part in [
    "saved_model.pb",
    "variables/variables.index",
    "variables/variables.data-00000-of-00001",
]:
    # get response content and save to disk in checkpoint path
    content = requests.get(f"{checkpoint_base_url}/{part}").content
    p = checkpoint_path / part
    p.parent.mkdir(parents=True, exist_ok=True)
    p.write_bytes(content)

print([p.as_posix() for p in checkpoint_path.glob("**/*")])

['data/raw/birdnet/v2.2/checkpoint/variables', 'data/raw/birdnet/v2.2/checkpoint/saved_model.pb', 'data/raw/birdnet/v2.2/checkpoint/variables/variables.index', 'data/raw/birdnet/v2.2/checkpoint/variables/variables.data-00000-of-00001']


In [5]:
import tensorflow as tf

model = tf.saved_model.load(checkpoint_path, tags=["serve"])

In [6]:
from tensorflow import keras

pb_model = keras.models.load_model(checkpoint_path)
pb_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x7fb3ee853d10>

In [13]:
import numpy as np


input_shape = pb_model.model.layers[0].get_config()["batch_input_shape"][1]
sample = np.random.rand(1, input_shape)

z = pb_model.model.predict(sample)
z.shape

1/1 [==============================] - 0s 230ms/step


(1, 3337)

- https://stackoverflow.com/questions/41711190/keras-how-to-get-the-output-of-each-layer/41712013
- https://stackoverflow.com/questions/52444175/get-the-before-last-feature-of-network-for-embedding-is-not-working

In [32]:
[layer.name for layer in pb_model.model.layers]

['INPUT',
 'ADVANCED_SPEC1',
 'BNORM_SPEC_NOQUANT',
 'CONV_0',
 'BNORM_0',
 'ACT_0',
 'pool_0_MAX',
 'pool_0_AVG',
 'pool_0_CONCAT',
 'pool_0_ACT_QUANT',
 'pool_0_CONV',
 'BLOCK_1-1_CONV_1',
 'BLOCK_1-1_BN_1',
 'BLOCK_1-1_ACT_1',
 'BLOCK_1-1_ACT_QUANT',
 'BLOCK_1-1_CONV_3',
 'BLOCK_1-1_BN_3',
 'BLOCK_1-2_CONV_1',
 'BLOCK_1-2_BN_1',
 'BLOCK_1-2_ACT_1',
 'BLOCK_1-2_ACT_QUANT',
 'BLOCK_1-2_CONV_3',
 'BLOCK_1-2_BN_3',
 'BLOCK_1-2_ADD',
 'BLOCK_1-3_CONV_1',
 'BLOCK_1-3_BN_1',
 'BLOCK_1-3_ACT_1',
 'BLOCK_1-3_ACT_QUANT',
 'BLOCK_1-3_CONV_3',
 'BLOCK_1-3_BN_3',
 'BLOCK_1-3_ADD',
 'BLOCK_2-1_CONV_1',
 'BLOCK_2-1_BN_1',
 'BLOCK_2-1_ACT_1',
 'BLOCK_2-1_ACT_QUANT',
 'BLOCK_2-1_CONV_3',
 'BLOCK_2-1_BN_3',
 'BLOCK_2-2_CONV_1',
 'BLOCK_2-2_BN_1',
 'BLOCK_2-2_ACT_1',
 'BLOCK_2-2_ACT_QUANT',
 'BLOCK_2-2_CONV_3',
 'BLOCK_2-2_BN_3',
 'BLOCK_2-2_ADD',
 'BLOCK_2-3_CONV_1',
 'BLOCK_2-3_BN_1',
 'BLOCK_2-3_ACT_1',
 'BLOCK_2-3_ACT_QUANT',
 'BLOCK_2-3_CONV_3',
 'BLOCK_2-3_BN_3',
 'BLOCK_2-3_ADD',
 'BLOCK_2-4_CO

In [37]:
from keras.models import Model

emb_model = Model(
    inputs=pb_model.model.input,
    outputs=[pb_model.model.get_layer("GLOBAL_AVG_POOL").output],
)

z = emb_model.predict(sample)
z.shape

1/1 [==============================] - 2s 2s/step


(1, 320)